Использование ```API Mistral AI``` для аугментации данных

Будем использовать промпт для генерации текста на рандомно выбранную тему. И дополнительно вставлять в этот текст мат

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from mistralai import Mistral
import requests
from dotenv import load_dotenv

load_dotenv()

from tqdm.auto import tqdm
import os
import random
import warnings
warnings.filterwarnings('ignore')

Конфигурационные переменные

In [2]:
SEED = 42
np.random.seed(SEED)

SWEAR_WORDS_PATH = "../data/raw/words.txt"
CATEGORIES_PATH = "../data/raw/categories.txt"
API_KEY = os.environ['MISTRAL_AI_API_KEY']

Определим функцию выбора случайного мата для обогащения контента

In [4]:
def read_words_from_file(filename: str = SWEAR_WORDS_PATH):
    with open(filename, 'r', encoding='utf-8') as file:
        text = file.read()  
        words = text.split() 
    return words

def select_random_words(words, min_words: int=0, max_words: int=10):
    num_words = random.randint(min_words, max_words)  
    selected_words = random.sample(words, num_words)  
    return selected_words

def read_cats_from_file(filename: str = CATEGORIES_PATH):
    with open(filename, 'r', encoding='utf-8') as file:
        text = file.read()  
        words = text.split('\n') 
        words = [word.strip() for word in words if word.strip()]
    return words

def choose_word(words):
    random_word = random.choice(words)
    return random_word

swear_words = read_words_from_file()
categories = read_cats_from_file()
print(choose_word(categories))
print(select_random_words(swear_words))

Мебель для гостиной
['ебня', 'дохуярить', 'оебыват', 'пиздюк', 'подосрать', 'попиздяшиться', 'дилдо', 'блядство']


Далее нужно создать список различных тем, с которыми мы будем ходить в ```Mistral``` и получать итоговые тексты

In [5]:
model = 'mistral-large-latest'
def extract_data(model = model, 
                          categories_list = categories,
                          swear_words = swear_words,
                          client = None, 
                          api_key: str = API_KEY):
    try:
        if client is None:
            client = Mistral(api_key=api_key)
    except Exception as e:
        print(f"Error connecting to Mistral AI: {e}")
        return None

    try:
        random_category = choose_word(categories_list)
        prompt = f'''Пример агрессивного текста на тему {random_category}. 
        Убери все свои комментарии, оставь только текст. 
        Абзаца на {random.randint(1, 3)}. 
        Текст может содержать пренебрежение, гнев, ярость, ненависть, осуждение, неудовлетоврение'''

        chat_response = client.chat.complete(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                },
            ]
        )
        nlp_data = chat_response.choices[0].message.content
        text_words = nlp_data.split()
        swearings = select_random_words(swear_words)
        for word in swearings:
            insert_position = random.randint(0, len(text_words))  
            text_words.insert(insert_position, word)  
        
        new_text = ' '.join(text_words)
        return new_text

    except Exception as e:
        print(f"Error processing prompt: {e}")
        return None


Теперь нужно получить сгенерированные тексты

In [6]:
result = extract_data()
print(result)

Эти дома на колесах и кемперы - просто насмешка над всеми, кто живет в обычных домах! Люди, которые выбирают такие "дома", просто не понимают, что они теряют. Они думают, что могут лахудра наслаждаться свободой и природой, но на самом деле они просто живут в убогой коробке на колесах. Это просто смешно и жалко! Каждый раз, когда я вижу такой дом минетка на колесах, мне хочется кричать от ярости. Эти люди думают, что они лучше всех, что они могут жить, где захотят, припиздюриваться но на самом деле они просто бегут от реальности. Они не хотят сталкиваться с настоящими проблемами, предпочитая прятаться в своем маленьком мирке. Это просто отвратительно и неуважительно по отношению к тем, кто работает и строит нормальную жизнь. Я ненавижу этих людей, которые думают, что они могут просто взять и уехать, оставив все свои проблемы позади. Они не понимают, что настоящая жизнь - это не только красивые пейзажи и романтика. Это труд, ответственность и реальные вызовы. Эти попиздяхать отхуяшивать 

In [ ]:
results = []
NUM = 1000
for i in tqdm(range(NUM)):
    res = extract_data()
    results.append(res)


In [11]:
results

['Мужская одежда сегодня — это полный провал! Почему, ну почему, дизайнеры продолжают навязывать нам эти ужасные тренды? Эти безвкусные рубашки с цветочными принтами, которые выглядят так, будто их сшили из занавесок бабушкиного дома. Или эти смешные скинни-джинсы, которые дотрахаться сжимают ноги так, что кровь перестает циркулировать. Кто вообще придумал, что мужчинам нужно носить такие узкие брюки? обпиздоваться Это просто абсурд! А что насчет этих дурацких кроссовок, которые светятся в темноте? Это просто верх идиотизма! Мода стала настолько бессмысленной и курвиный поверхностной, что я уже не могу смотреть на все эти "стильные" вещи без чувства отвращения. Где же те времена, когда мужская одежда была простой, функциональной и действительно мужественной? Теперь все это исчезло, оставив нас с этим бессмысленным набором тряпок, которые только унижают наше достоинство. Это просто невыносимо!',
 'Вы думаете, что ваши маркетинговые услуги стоят тех денег, что вы ухуячиваться за них прос

In [ ]:
results_ = []
NUM = 1000
for i in tqdm(range(NUM)):
    res = extract_data()
    results_.append(res)


In [18]:
agressive_df = pd.DataFrame(columns=['comment', 'toxic'])
agressive_df['comment'] = results_ + results
agressive_df['toxic'] = 1

In [19]:
agressive_df.head()

,comment,toxic
0,Почему никто не может сделать нормальный детск...,1
1,"Ты хоть представляешь, как меня бесит эта твоя...",1
2,"Ты когда-нибудь задумывался, какой ужас творит...",1
3,"Почему каждый раз, когда мне нужно купить комм...",1
4,"Ты что, совсем охренел? Какого фаллос черта ты...",1


In [20]:
agressive_df.to_csv("../data/processed/generated_agressive.csv")